# sumaryzacja tekstu

In [1]:
from google.colab import drive


drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
!nvidia-smi

Sat Nov 15 16:06:05 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   79C    P0             85W /   70W |    5598MiB /  15360MiB |     62%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
# 1. Force uninstall both to clear the conflict
!pip uninstall -y unsloth unsloth_zoo

# 2. Install Unsloth Zoo DIRECTLY from Git (to get 'tiled_mlp')
!pip install --upgrade --no-cache-dir "unsloth_zoo @ git+https://github.com/unslothai/unsloth-zoo.git"

# 3. Install Unsloth main library from Git
!pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

Found existing installation: unsloth 2025.11.3
Uninstalling unsloth-2025.11.3:
  Successfully uninstalled unsloth-2025.11.3
Found existing installation: unsloth_zoo 2025.11.4
Uninstalling unsloth_zoo-2025.11.4:
  Successfully uninstalled unsloth_zoo-2025.11.4
  Cloning https://github.com/unslothai/unsloth-zoo.git to /tmp/pip-install-icwdep9b/unsloth-zoo_28ebe30050b84ecbb28b83333ce19da4
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth-zoo.git /tmp/pip-install-icwdep9b/unsloth-zoo_28ebe30050b84ecbb28b83333ce19da4
  Resolved https://github.com/unslothai/unsloth-zoo.git to commit b60fd19e4e1c27014553f7194669d66cdb96db2c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 33.4 MB/s eta 0:00:00
  Created wheel for unsloth_zoo: filename=unsloth_zoo-2025.11.4-py3-none-any.whl size=281896 sha256=cb1d23cc9c930a20b

In [ ]:
!pip install "datasets<3.0.0"


  Using cached datasets-2.21.0-py3-none-any.whl.metadata (21 kB)
Using cached datasets-2.21.0-py3-none-any.whl (527 kB)
  Attempting uninstall: datasets
    Found existing installation: datasets 4.3.0
    Uninstalling datasets-4.3.0:
      Successfully uninstalled datasets-4.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
trl 0.24.0 requires datasets>=3.0.0, but you have datasets 2.21.0 which is incompatible.
unsloth-zoo 2025.11.4 requires datasets!=4.0.*,!=4.1.0,<4.4.0,>=3.4.1, but you have datasets 2.21.0 which is incompatible.


In [ ]:
!pip install bert-extractive-summarizer

In [ ]:
!pip install pytorch-ignite
!pip install torchmetrics

## importowanie datesetu

In [ ]:
from datasets import load_dataset


ds = load_dataset("xsum", trust_remote_code=True)
ds= ds.filter(lambda example: len(example["document"]) < 750)
split_result = ds["test"].train_test_split(test_size=2, seed=42)
shots = split_result["test"]
ds["test"] = split_result["train"]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


In [ ]:
print(ds)
print(shots)

DatasetDict({
    train: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 32053
    })
    validation: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 1746
    })
    test: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 1727
    })
})
Dataset({
    features: ['document', 'summary', 'id'],
    num_rows: 2
})


In [ ]:
from summarizer import Summarizer
model = Summarizer()

In [ ]:
from ignite.metrics import Rouge

m = Rouge(variants=["L", 2], multiref="best")

candidate = "the cat is not there".split()
references = [
    "the cat is on the mat".split(),
    "there is a cat on the mat".split()
]

m.update(([candidate], [references]))

print(m.compute())

{'Rouge-L-P': 0.6, 'Rouge-L-R': 0.5, 'Rouge-L-F': 0.5, 'Rouge-2-P': 0.5, 'Rouge-2-R': 0.4, 'Rouge-2-F': 0.4}


In [ ]:
from pprint import pprint
from torchmetrics.text.bert import BERTScore
preds = """
The full cost of damage in Newton Stewart, one of the areas worst affected, is still being assessed.
Repair work is ongoing in Hawick and many roads in Peeblesshire remain badly affected by standing water.
Trains on the west coast mainline face disruption due to damage at the Lamington Viaduct.
Many businesses and householders were affected by flooding in Newton Stewart after the River Cree overflowed into the town.
First Minister Nicola Sturgeon visited the area to inspect the damage.
The waters breached a retaining wall, flooding many commercial properties on Victoria Street - the main shopping thoroughfare.
Jeanette Tate, who owns the Cinnamon Cafe which was badly affected, said she could not fault the multi-agency response once the flood hit.
However, she said more preventative work could have been carried out to ensure the retaining wall did not fail.
"It is difficult but I do think there is so much publicity for Dumfries and the Nith - and I totally appreciate that - but it is almost like we're neglected or forgotten," she said.
"That may not be true but it is perhaps my perspective over the last few days.
"Why were you not ready to help us a bit more when the warning and the alarm alerts had gone out?"
Meanwhile, a flood alert remains in place across the Borders because of the constant rain.
Peebles was badly hit by problems, sparking calls to introduce more defences in the area.
Scottish Borders Council has put a list on its website of the roads worst affected and drivers have been urged not to ignore closure signs.
The Labour Party's deputy Scottish leader Alex Rowley was in Hawick on Monday to see the situation first hand.
He said it was important to get the flood protection plan right but backed calls to speed up the process.
"I was quite taken aback by the amount of damage that has been done," he said.
"Obviously it is heart-breaking for people who have been forced out of their homes and the impact on businesses."
He said it was important that "immediate steps" were taken to protect the areas most vulnerable and a clear timetable put in place for flood prevention plans.
Have you been affected by flooding in Dumfries and Galloway or the Borders? Tell us about your experience of the situation and how it was handled. Email us on selkirk.news@bbc.co.uk or dumfries@bbc.co.uk.
"""
result = model(preds)
# print(result)
target ="""
Clean-up operations are continuing across the Scottish Borders and Dumfries and Galloway after flooding caused by Storm Frank.
"""
print(f'target: {target}')
print(f'result: {result}')
bertscore = BERTScore(model_name_or_path="microsoft/deberta-v3-large", lang='en')
pprint(bertscore([result], [target]))

target: 
Clean-up operations are continuing across the Scottish Borders and Dumfries and Galloway after flooding caused by Storm Frank.

result: The full cost of damage in Newton Stewart, one of the areas worst affected, is still being assessed. Trains on the west coast mainline face disruption due to damage at the Lamington Viaduct. First Minister Nicola Sturgeon visited the area to inspect the damage. "That may not be true but it is perhaps my perspective over the last few days. He said it was important to get the flood protection plan right but backed calls to speed up the process.


/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
The following layers were not sharded: encoder.layer.*.output.LayerNorm.bias, encoder.LayerNorm.bias, encoder.layer.*.attention.self.query_proj.bias, encoder.layer.*.attention.output.dense.weight, encoder.rel_embeddings.weight, encoder.layer.*.output.dense.weight, encoder.layer.*.output.LayerNorm.weight, encoder.layer.*.attention.self.value_proj.bias, encoder.layer.*.attention.self.query_proj.weight, encoder.layer.*.attention.self.key_proj.bias, encoder.layer.*.intermediate.dense.weight, embedding

{'f1': tensor(0.7959), 'precision': tensor(0.7827), 'recall': tensor(0.8095)}


In [ ]:
num_workers = 1
batch_size = 1

In [ ]:
from torch.utils.data import DataLoader
import torch
test_dataset = ds["test"]
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
print(test_dataset)


Dataset({
    features: ['document', 'summary', 'id'],
    num_rows: 1727
})


In [ ]:
def bert_score_loss(batch, model, bert: bool, tokenizer: any, prompt = None, promptsuff=None):
    if bert:
        predictions = [model(doc, ratio=0.1) for doc in batch['document']]
    else:
        docs = [doc for doc in batch['document']]
        prompted  = [f'{prompt}{doc}{promptsuff}'for doc in batch['document']]
        inputs = tokenizer(
        prompted,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=4096
    )
        device = next(model.parameters()).device
        inputs = {k: v.to(device) for k, v in inputs.items()}
        summary_ids = model.generate(
        inputs['input_ids'],
        attention_mask=inputs['attention_mask'],
        max_new_tokens=50,
        do_sample=True,
        top_k=0,
        temperature=0.5,
    )
        input_length = inputs["input_ids"].shape[1]
        new_tokens = summary_ids[:, input_length:]
        predictions = tokenizer.batch_decode(
            new_tokens, skip_special_tokens=True
        )
    print(f"Doc: {batch['document'][0][:100]}")
    print(f"Prediction: {predictions[0]}")
    print(f"Target: {batch['summary'][0]}")
    predictions = [pred if pred.strip() else "." for pred in predictions]
    for i, pred in enumerate(predictions):
        if pred.strip() == "":
            print("EMPTY PREDICTION")
            print(batch['document'][i])
    return bertscore(predictions, batch["summary"])

In [ ]:
def batch_to_device(batch: dict, device) -> dict:
    return {k: v.to(device) for k, v in batch.items()}

In [ ]:
import torch
import torch.nn as nn
from torch import Tensor
def calc_loss_batch(batch: dict, model: nn.Module) -> Tensor:
    logits = model(batch['input_ids'])
    loss = torch.nn.functional.cross_entropy(logits.flatten(0, 1), batch['labels'].flatten())
    return loss

In [ ]:
def calc_loss_loader(data_loader, model, device, num_batches: int, bert: bool, tokenizer: any,prompt = None, promptsuff = None) -> float:
    total_loss = {'precision': [], 'recall': [], 'f1': []}
    for i, batch in enumerate(data_loader):
        if i >= num_batches:
            break
        # batch = batch_to_device(batch, device)
        loss = bert_score_loss(batch, model, bert, tokenizer, prompt, promptsuff)
        total_loss['precision'].append(loss['precision'].unsqueeze(0))
        total_loss['recall'].append(loss['recall'].unsqueeze(0))
        total_loss['f1'].append(loss['f1'].unsqueeze(0))
    total_loss['precision'] = torch.cat(total_loss['precision'])
    total_loss['recall'] = torch.cat(total_loss['recall'])
    total_loss['f1'] = torch.cat(total_loss['f1'])
    print(total_loss)
    return (sum(total_loss['precision'])/ len(total_loss['precision']),sum(total_loss['recall'])/ len(total_loss['recall']),sum(total_loss['f1'])/ len(total_loss['f1']))

In [ ]:
def evaluate_model(model: nn.Module, loader: DataLoader,
                    device, eval_iter: int, bert: bool, tokenizer: any, prompt="", promptsuff ="") -> tuple[float, float]:
    if not bert:
      model.eval()
    with torch.no_grad():
        loss = calc_loss_loader(loader, model, device, num_batches=eval_iter, bert=bert, tokenizer=tokenizer, prompt=prompt, promptsuff=promptsuff)
    return loss

In [ ]:
from torch import nn
loss = evaluate_model(model, test_loader, device='cuda', eval_iter=1, bert=True, tokenizer=None)
print(loss)



Doc: The Faraday Lecture Block dates back to the 1960s and the university says its "tired and monotonous 
Prediction: The Faraday Lecture Block dates back to the 1960s and the university says its "tired and monotonous design" means it has become a bit of an eyesore. The university said the new look would add vibrancy to the building and the mall area.
Target: One of the main buildings on Swansea University's Singleton Park campus is to undergo a £2.75m makeover.


/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
The following layers were not sharded: encoder.layer.*.output.LayerNorm.bias, encoder.LayerNorm.bias, encoder.layer.*.attention.self.query_proj.bias, encoder.layer.*.attention.output.dense.weight, encoder.rel_embeddings.weight, encoder.layer.*.output.dense.weight, encoder.layer.*.output.LayerNorm.weight, encoder.layer.*.attention.self.value_proj.bias, encoder.layer.*.attention.self.query_proj.weight, encoder.layer.*.attention.self.key_proj.bias, encoder.layer.*.intermediate.dense.weight, embedding

{'precision': tensor([0.7970]), 'recall': tensor([0.8037]), 'f1': tensor([0.8003])}
(tensor(0.7970), tensor(0.8037), tensor(0.8003))


In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")
tokenizer.padding_side = 'left'
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    # Some models also require explicitly setting the pad_token_id
    tokenizer.pad_token_id = tokenizer.eos_token_id
model2 = AutoModelForCausalLM.from_pretrained("openai-community/gpt2")

The following layers were not sharded: transformer.h.*.mlp.c_proj.weight, transformer.h.*.ln_*.weight, transformer.h.*.ln_*.bias, lm_head.weight, transformer.h.*.attn.c_proj.bias, transformer.h.*.attn.c_proj.weight, transformer.wte.weight, transformer.h.*.attn.c_attn.bias, transformer.wpe.weight, transformer.h.*.mlp.c_fc.weight, transformer.h.*.mlp.c_fc.bias, transformer.ln_f.bias, transformer.h.*.mlp.c_proj.bias, transformer.ln_f.weight, transformer.h.*.attn.c_attn.weight


In [ ]:
def generate_prompt(tokenizer, size):
    shots_loader = DataLoader(shots, batch_size=100000, shuffle=True, num_workers=num_workers)
    for batch in shots_loader:
        for i in range(len(batch['document'])):
            prompt += f'{i+1}. {batch['document'][i]}\n###Summary\n{batch['summary'][i]}'
            prompt += tokenizer.eos_token
    return prompt

prompt = generate_prompt(tokenizer, 3)
print(prompt)

NameError: name 'tokenizer' is not defined

In [ ]:
summary = "\n###Summary\n"

In [ ]:
print(prompt)
print(evaluate_model(model2, test_loader, device='cuda', eval_iter=1, bert=False, tokenizer=tokenizer, prompt = prompt, promptsuff=summary))


Summarize.  What are these texts about? 1. Euro 2016 winners Portugal will now face Chile in the last four on Wednesday (19:00 BST).
Australia went ahead through winger James Troisi, before substitute Martin Rodriguez levelled from close range.
World champions Germany moved above Chile with a 3-1 win over Cameroon and will play Mexico on Thursday (19:00).
RB Leipzig forward Timo Werner scored twice, with Hoffenheim midfielder Kerem Demirbay also on target for the experimental German squad competing in Russia.
Porto forward Vincent Aboubakar netted for Cameroon, who finished bottom of the group, with a draw from their match against Australia their only point.
Group A:
Group B:
###Summary
Chile qualified behind Germany from Group B to reach the Confederations Cup semi-finals after drawing 1-1 with Australia in their final group game.<|endoftext|>2. Several fire appliances were called to the scene at Carnsalloch House to tackle the blaze.
One witness reported damage to the main part of th

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Doc: McKenzie, 38, will take on Jahmaine Smyle at York Hall on 12 November for the English super-middlewe
Prediction: McKenzie's father, former Manchester United and England striker Peter McKenzie, is a former United player and has also been involved in the sport of rugby.
McKenzie, who is currently in the process of signing a new deal with the Scottish
Target: Former Premier League striker Leon McKenzie is to fight for a national boxing title.


The following layers were not sharded: encoder.layer.*.output.LayerNorm.weight, encoder.layer.*.attention.self.query_proj.bias, encoder.layer.*.attention.output.LayerNorm.weight, encoder.layer.*.intermediate.dense.bias, encoder.layer.*.attention.output.dense.bias, encoder.layer.*.intermediate.dense.weight, encoder.layer.*.attention.output.LayerNorm.bias, embeddings.LayerNorm.bias, encoder.layer.*.attention.self.value_proj.bias, encoder.layer.*.output.LayerNorm.bias, embeddings.word_embeddings.weight, encoder.layer.*.output.dense.weight, encoder.layer.*.attention.output.dense.weight, embeddings.LayerNorm.weight, encoder.layer.*.attention.self.key_proj.bias, encoder.layer.*.attention.self.key_proj.weight, encoder.layer.*.output.dense.bias, encoder.LayerNorm.bias, encoder.layer.*.attention.self.query_proj.weight, encoder.LayerNorm.weight, encoder.rel_embeddings.weight, encoder.layer.*.attention.self.value_proj.weight


{'precision': tensor([0.7703]), 'recall': tensor([0.8087]), 'f1': tensor([0.7891])}
(tensor(0.7703), tensor(0.8087), tensor(0.7891))


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer3 = AutoTokenizer.from_pretrained("Qwen/Qwen1.5-0.5B")
tokenizer3.padding_side = 'left'
if tokenizer3.pad_token is None:
    tokenizer3.pad_token = tokenizer3.eos_token
    # Some models also require explicitly setting the pad_token_id
    tokenizer3.pad_token_id = tokenizer3.eos_token_id
model3 = AutoModelForCausalLM.from_pretrained("Qwen/Qwen1.5-0.5B")




tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.24G [00:00<?, ?B/s]

The following TP rules were not applied on any of the layers: {'layers.*.self_attn.q_proj': 'colwise', 'layers.*.self_attn.k_proj': 'colwise', 'layers.*.self_attn.v_proj': 'colwise', 'layers.*.self_attn.o_proj': 'rowwise', 'layers.*.mlp.gate_proj': 'colwise', 'layers.*.mlp.up_proj': 'colwise', 'layers.*.mlp.down_proj': 'rowwise'}
The following layers were not sharded: model.norm.weight, model.layers.*.input_layernorm.weight, model.layers.*.self_attn.v_proj.bias, model.layers.*.post_attention_layernorm.weight, model.layers.*.self_attn.k_proj.weight, lm_head.weight, model.layers.*.self_attn.q_proj.bias, model.embed_tokens.weight


generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

In [ ]:
prompt3 = generate_prompt(tokenizer3, 0)
print(evaluate_model(model3, test_loader, device='cuda', eval_iter=1, bert=False, tokenizer=tokenizer3, prompt=prompt3, promptsuff = summary))

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Doc: The 26-year-old ex-Dagenham and York defender started all but four of Wood's league matches last sea
Prediction:  The former Manchester City striker joined the club for £825,000 in January. The former Manchester City striker joined the club for £825,000 in January. The former Manchester City striker joined the club for £
Target: Dover Athletic have signed Femi Ilesanmi on a one-year contract after the left-back's departure from National League rivals Boreham Wood.


The following layers were not sharded: encoder.layer.*.output.LayerNorm.weight, encoder.layer.*.attention.self.query_proj.bias, encoder.layer.*.attention.output.LayerNorm.weight, encoder.layer.*.intermediate.dense.bias, encoder.layer.*.attention.output.dense.bias, encoder.layer.*.intermediate.dense.weight, encoder.layer.*.attention.output.LayerNorm.bias, embeddings.LayerNorm.bias, encoder.layer.*.attention.self.value_proj.bias, encoder.layer.*.output.LayerNorm.bias, embeddings.word_embeddings.weight, encoder.layer.*.output.dense.weight, encoder.layer.*.attention.output.dense.weight, embeddings.LayerNorm.weight, encoder.layer.*.attention.self.key_proj.bias, encoder.layer.*.attention.self.key_proj.weight, encoder.layer.*.output.dense.bias, encoder.LayerNorm.bias, encoder.layer.*.attention.self.query_proj.weight, encoder.LayerNorm.weight, encoder.rel_embeddings.weight, encoder.layer.*.attention.self.value_proj.weight


{'precision': tensor([0.6278]), 'recall': tensor([0.6008]), 'f1': tensor([0.6140])}
(tensor(0.6278), tensor(0.6008), tensor(0.6140))


In [ ]:
prompt4 = "Summarize this text. Don't hallucinate. Be thorough and explicit. Always provide an anwer What is this text about? "
print(
    evaluate_model(
        model3,
        test_loader,
        device="cuda",
        eval_iter=1,
        bert=False,
        tokenizer=tokenizer3,
        prompt=prompt4,
    )
)

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Doc: The Faraday Lecture Block dates back to the 1960s and the university says its "tired and monotonous 
Prediction:  The entrance hall will have a new design, with a glass-clad entrance with a double-height space. The main building will have new glass walls, with a new entrance hall and a second floor extension. It will also have improved access, including a
Target: One of the main buildings on Swansea University's Singleton Park campus is to undergo a £2.75m makeover.


The following layers were not sharded: encoder.layer.*.output.LayerNorm.weight, encoder.layer.*.attention.self.query_proj.bias, encoder.layer.*.attention.output.LayerNorm.weight, encoder.layer.*.intermediate.dense.bias, encoder.layer.*.attention.output.dense.bias, encoder.layer.*.intermediate.dense.weight, encoder.layer.*.attention.output.LayerNorm.bias, embeddings.LayerNorm.bias, encoder.layer.*.attention.self.value_proj.bias, encoder.layer.*.output.LayerNorm.bias, embeddings.word_embeddings.weight, encoder.layer.*.output.dense.weight, encoder.layer.*.attention.output.dense.weight, embeddings.LayerNorm.weight, encoder.layer.*.attention.self.key_proj.bias, encoder.layer.*.attention.self.key_proj.weight, encoder.layer.*.output.dense.bias, encoder.LayerNorm.bias, encoder.layer.*.attention.self.query_proj.weight, encoder.LayerNorm.weight, encoder.rel_embeddings.weight, encoder.layer.*.attention.self.value_proj.weight


{'precision': tensor([0.7874]), 'recall': tensor([0.7596]), 'f1': tensor([0.7733])}
(tensor(0.7874), tensor(0.7596), tensor(0.7733))


In [ ]:
print(
    evaluate_model(
        model2,
        test_loader,
        device="cuda",
        eval_iter=2,
        bert=False,
        tokenizer=tokenizer,
        prompt=prompt4,
    )
)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Doc: Bloemfontein joins Beijing and Lisbon in being suspended from carrying out any Wada-related anti-dop
Prediction: 
Bloemfontein's legal team has argued that the suspension should be lifted because it is the first time the Wada-accredited labs have been suspended.
Wada said it would appeal the decision.
"We are very
Target: South Africa's Doping Control Laboratory has become the fourth lab to be sanctioned by the World Anti-Doping Agency (Wada) in the past month.


The following layers were not sharded: encoder.layer.*.output.LayerNorm.weight, encoder.layer.*.attention.self.query_proj.bias, encoder.layer.*.attention.output.LayerNorm.weight, encoder.layer.*.intermediate.dense.bias, encoder.layer.*.attention.output.dense.bias, encoder.layer.*.intermediate.dense.weight, encoder.layer.*.attention.output.LayerNorm.bias, embeddings.LayerNorm.bias, encoder.layer.*.attention.self.value_proj.bias, encoder.layer.*.output.LayerNorm.bias, embeddings.word_embeddings.weight, encoder.layer.*.output.dense.weight, encoder.layer.*.attention.output.dense.weight, embeddings.LayerNorm.weight, encoder.layer.*.attention.self.key_proj.bias, encoder.layer.*.attention.self.key_proj.weight, encoder.layer.*.output.dense.bias, encoder.LayerNorm.bias, encoder.layer.*.attention.self.query_proj.weight, encoder.LayerNorm.weight, encoder.rel_embeddings.weight, encoder.layer.*.attention.self.value_proj.weight
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Doc: Media playback is not supported on this device
Dusmatov, 23, was awarded a unanimous points victory 
Prediction: 
Target: Uzbekistan's Hasanboy Dusmatov won Olympic gold in the men's light-flyweight with victory over Colombian Yuberjen Herney Martinez.


The following layers were not sharded: encoder.layer.*.output.LayerNorm.weight, encoder.layer.*.attention.self.query_proj.bias, encoder.layer.*.attention.output.LayerNorm.weight, encoder.layer.*.intermediate.dense.bias, encoder.layer.*.attention.output.dense.bias, encoder.layer.*.intermediate.dense.weight, encoder.layer.*.attention.output.LayerNorm.bias, embeddings.LayerNorm.bias, encoder.layer.*.attention.self.value_proj.bias, encoder.layer.*.output.LayerNorm.bias, embeddings.word_embeddings.weight, encoder.layer.*.output.dense.weight, encoder.layer.*.attention.output.dense.weight, embeddings.LayerNorm.weight, encoder.layer.*.attention.self.key_proj.bias, encoder.layer.*.attention.self.key_proj.weight, encoder.layer.*.output.dense.bias, encoder.LayerNorm.bias, encoder.layer.*.attention.self.query_proj.weight, encoder.LayerNorm.weight, encoder.rel_embeddings.weight, encoder.layer.*.attention.self.value_proj.weight


{'precision': tensor([0.8011, 0.4315]), 'recall': tensor([0.7581, 0.1686]), 'f1': tensor([0.7790, 0.2425])}
(tensor(0.6163), tensor(0.4634), tensor(0.5107))


In [ ]:
import torch

print(torch.cuda.is_available())

True


In [ ]:
# !pip install -U bitsandbytes

In [ ]:
from unsloth import FastLanguageModel

ft_model, ft_tokenzier = FastLanguageModel.from_pretrained(
    model_name="Qwen/Qwen1.5-0.5B", max_seq_length=4096, load_in_4bit=False
)

==((====))==  Unsloth 2025.11.3: Fast Qwen2 patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


The following TP rules were not applied on any of the layers: {'layers.*.self_attn.q_proj': 'colwise', 'layers.*.self_attn.k_proj': 'colwise', 'layers.*.self_attn.v_proj': 'colwise', 'layers.*.self_attn.o_proj': 'rowwise', 'layers.*.mlp.gate_proj': 'colwise', 'layers.*.mlp.up_proj': 'colwise', 'layers.*.mlp.down_proj': 'rowwise'}
The following layers were not sharded: model.norm.weight, model.layers.*.input_layernorm.weight, model.layers.*.self_attn.v_proj.bias, model.layers.*.post_attention_layernorm.weight, model.layers.*.self_attn.k_proj.weight, lm_head.weight, model.layers.*.self_attn.q_proj.bias, model.embed_tokens.weight


Qwen/Qwen1.5-0.5B does not have a padding token! Will use pad_token = <|PAD_TOKEN|>.


In [ ]:
ft_ds = ds


def merge_columns(example):
    example["text"] = (
        f"{example['document']}\n### Summary\n{example['summary']}"
    )
    return example

ft_ds = ft_ds.map(merge_columns)

In [ ]:
from trl import SFTTrainer, SFTConfig
ft_model = FastLanguageModel.get_peft_model(
    ft_model,
    r=32,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                  "gate_proj", "up_proj", "down_proj",],
    lora_alpha=16,
    lora_dropout=0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state=1234,
    use_rslora = False,
    loftq_config = None,

)
trainer = SFTTrainer(
    model=ft_model,
    train_dataset=ft_ds["train"],
    eval_dataset=ft_ds["validation"],
    tokenizer=ft_tokenzier,
    args=SFTConfig(
        max_seq_length=4096,
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=10,
        max_steps=4000,
        logging_steps=1,
        output_dir="outputs",
        optim="adamw_8bit",
        seed=3407,
        eval_strategy="steps",
        eval_steps=200,
    ),
)
trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 32,053 | Num Epochs = 1 | Total steps = 4,000
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 15,138,816 of 479,126,528 (3.16% trained)


Step,Training Loss,Validation Loss
200,2.381800,2.497168


: 

In [ ]:
evaluate_model(ft_model, test_loader, 'cuda', 50, bert=False, tokenizer = ft_tokenzier, promptsuff = "\n### Summary\n")

Doc: A party spokesperson said the timing made it "impractical" to put together and finance "a meaningful
Prediction: The Conservatives have decided to not field candidates in the upcoming Cornwall Council elections. The party will be campaigning for the next election in the 2015 general election.
Target: Mebyon Kernow, which campaigns for a National Assembly for Cornwall, has announced it will not be putting forward any candidates for the 2017 general election.


The following layers were not sharded: encoder.layer.*.output.LayerNorm.weight, encoder.layer.*.attention.self.query_proj.bias, encoder.layer.*.attention.output.LayerNorm.weight, encoder.layer.*.intermediate.dense.bias, encoder.layer.*.attention.output.dense.bias, encoder.layer.*.intermediate.dense.weight, encoder.layer.*.attention.output.LayerNorm.bias, embeddings.LayerNorm.bias, encoder.layer.*.attention.self.value_proj.bias, encoder.layer.*.output.LayerNorm.bias, embeddings.word_embeddings.weight, encoder.layer.*.output.dense.weight, encoder.layer.*.attention.output.dense.weight, embeddings.LayerNorm.weight, encoder.layer.*.attention.self.key_proj.bias, encoder.layer.*.attention.self.key_proj.weight, encoder.layer.*.output.dense.bias, encoder.LayerNorm.bias, encoder.layer.*.attention.self.query_proj.weight, encoder.LayerNorm.weight, encoder.rel_embeddings.weight, encoder.layer.*.attention.self.value_proj.weight


Doc: Two were picked up by a coastguard helicopter high up on the 3,196ft (974 metre) Munro.
Lomond Mount
Prediction: Two people have been rescued after being swept by a strong wind on a mountain in Scotland. The rescue team has said the winds were up to 80mph and there was snow on the mountain.
Target: Four hillwalkers who got into difficulties during severe weather on Ben Lomond have been rescued.


The following layers were not sharded: encoder.layer.*.output.LayerNorm.weight, encoder.layer.*.attention.self.query_proj.bias, encoder.layer.*.attention.output.LayerNorm.weight, encoder.layer.*.intermediate.dense.bias, encoder.layer.*.attention.output.dense.bias, encoder.layer.*.intermediate.dense.weight, encoder.layer.*.attention.output.LayerNorm.bias, embeddings.LayerNorm.bias, encoder.layer.*.attention.self.value_proj.bias, encoder.layer.*.output.LayerNorm.bias, embeddings.word_embeddings.weight, encoder.layer.*.output.dense.weight, encoder.layer.*.attention.output.dense.weight, embeddings.LayerNorm.weight, encoder.layer.*.attention.self.key_proj.bias, encoder.layer.*.attention.self.key_proj.weight, encoder.layer.*.output.dense.bias, encoder.LayerNorm.bias, encoder.layer.*.attention.self.query_proj.weight, encoder.LayerNorm.weight, encoder.rel_embeddings.weight, encoder.layer.*.attention.self.value_proj.weight


Doc: She will talk about her life outside of politics, including television appearances on Strictly Come 
Prediction: The BBC's Strictly Come Dancing is set to return to the UK festival circuit for the third time. The show's former host, Ronnie Browne, will be the star guest at the 2017 festival. Read on for all the
Target: Former Conservative politician Ann Widdecombe will headline this year's Nairn Book and Arts Festival.


...

The following layers were not sharded: encoder.layer.*.output.LayerNorm.weight, encoder.layer.*.attention.self.query_proj.bias, encoder.layer.*.attention.output.LayerNorm.weight, encoder.layer.*.intermediate.dense.bias, encoder.layer.*.attention.output.dense.bias, encoder.layer.*.intermediate.dense.weight, encoder.layer.*.attention.output.LayerNorm.bias, embeddings.LayerNorm.bias, encoder.layer.*.attention.self.value_proj.bias, encoder.layer.*.output.LayerNorm.bias, embeddings.word_embeddings.weight, encoder.layer.*.output.dense.weight, encoder.layer.*.attention.output.dense.weight, embeddings.LayerNorm.weight, encoder.layer.*.attention.self.key_proj.bias, encoder.layer.*.attention.self.key_proj.weight, encoder.layer.*.output.dense.bias, encoder.LayerNorm.bias, encoder.layer.*.attention.self.query_proj.weight, encoder.LayerNorm.weight, encoder.rel_embeddings.weight, encoder.layer.*.attention.self.value_proj.weight


Doc: The 25-year-old steps up to the National League after 22 appearances for Rushall this season.
He is 
Prediction: Rushall Town have signed forward Ben Mawson and defender Jean-Yves Koue Niate from Mansfield Town.
Target: Solihull Moors have signed centre-back Joel Kettle from Northern League Premier leaders Rushall Olympic.


The following layers were not sharded: encoder.layer.*.output.LayerNorm.weight, encoder.layer.*.attention.self.query_proj.bias, encoder.layer.*.attention.output.LayerNorm.weight, encoder.layer.*.intermediate.dense.bias, encoder.layer.*.attention.output.dense.bias, encoder.layer.*.intermediate.dense.weight, encoder.layer.*.attention.output.LayerNorm.bias, embeddings.LayerNorm.bias, encoder.layer.*.attention.self.value_proj.bias, encoder.layer.*.output.LayerNorm.bias, embeddings.word_embeddings.weight, encoder.layer.*.output.dense.weight, encoder.layer.*.attention.output.dense.weight, embeddings.LayerNorm.weight, encoder.layer.*.attention.self.key_proj.bias, encoder.layer.*.attention.self.key_proj.weight, encoder.layer.*.output.dense.bias, encoder.LayerNorm.bias, encoder.layer.*.attention.self.query_proj.weight, encoder.LayerNorm.weight, encoder.rel_embeddings.weight, encoder.layer.*.attention.self.value_proj.weight


{'precision': tensor([0.8212, 0.8061, 0.7640, 0.8180, 0.8248, 0.8291, 0.7962, 0.8670, 0.7887,
        0.8303]), 'recall': tensor([0.7902, 0.8263, 0.7947, 0.8214, 0.7781, 0.8551, 0.7908, 0.8793, 0.8032,
        0.8361]), 'f1': tensor([0.8054, 0.8161, 0.7790, 0.8197, 0.8008, 0.8419, 0.7935, 0.8731, 0.7959,
        0.8332])}


(tensor(0.8146), tensor(0.8175), tensor(0.8159))

In [ ]:
ft_model.save('ft')